In [ ]:
!pip install librosa pandas matplotlib seaborn soundfile jiwer \
transformers torchaudio wordcloud datasets scikit-learn


In [ ]:
!pip install fsspec==2025.3.2


In [ ]:
import os
import tarfile
import urllib.request

# Create a directory for the dataset
dataset_url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
dataset_dir = "/content/LibriSpeech"
archive_path = "/content/dev-clean.tar.gz"

# Download the dataset
urllib.request.urlretrieve(dataset_url, archive_path)

# Extract the dataset
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(path=dataset_dir)

print("Dataset extracted to:", dataset_dir)
import os
import tarfile
import urllib.request

# Create a directory for the dataset
dataset_url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
dataset_dir = "/content/LibriSpeech"
archive_path = "/content/dev-clean.tar.gz"

# Download the dataset
urllib.request.urlretrieve(dataset_url, archive_path)

# Extract the dataset
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(path=dataset_dir)

print("Dataset extracted to:", dataset_dir)


In [ ]:
import glob

# List a few .flac audio files
flac_files = sorted(glob.glob(dataset_dir + "/LibriSpeech/dev-clean/**/**/*.flac", recursive=True))
print("Number of audio files:", len(flac_files))
print("Sample file path:", flac_files[0])


In [ ]:
import soundfile as sf

# Convert first .flac to .wav
wav_output = "/content/sample.wav"
data, samplerate = sf.read(flac_files[0])
sf.write(wav_output, data, samplerate)
print("Converted to WAV:", wav_output)


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

# Load the audio
y, sr = librosa.load(wav_output, sr=16000)

# Display audio
Audio(wav_output)

# Waveform
plt.figure(figsize=(10, 3))
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform')
plt.show()


In [ ]:
!pip install librosa soundfile tqdm --quiet


In [ ]:
import os
import tarfile
import urllib.request

url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
data_dir = "/content/LibriSpeech"
os.makedirs(data_dir, exist_ok=True)

# Download
archive_path = os.path.join(data_dir, "dev-clean.tar.gz")
urllib.request.urlretrieve(url, archive_path)

# Extract
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(path=data_dir)

print("✅ Dataset ready.")


In [ ]:
import glob
import pandas as pd

wav_paths = []
transcripts = []

# Look inside all folders
for root, _, files in os.walk(f"{data_dir}/LibriSpeech/dev-clean"):
    trans_files = [f for f in files if f.endswith(".trans.txt")]
    for tf in trans_files:
        trans_path = os.path.join(root, tf)
        with open(trans_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split(" ", 1)
                file_id = parts[0]
                transcript = parts[1]
                flac_path = os.path.join(root, file_id + ".flac")
                if os.path.exists(flac_path):
                    wav_paths.append(flac_path)
                    transcripts.append(transcript)

df = pd.DataFrame({"wav_path": wav_paths, "transcript": transcripts})
print(f"✅ Found {len(df)} audio files.")
df.head()


In [ ]:
import librosa
import numpy as np

def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T  # time x features

# Test MFCC extraction
sample_mfcc = extract_mfcc(df.iloc[0]['wav_path'])
print("MFCC shape:", sample_mfcc.shape)


In [ ]:
import librosa
import numpy as np

def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T  # transpose to shape: (time, features)

# Example for one file
mfcc_features = extract_mfcc(df.iloc[0]['wav_path'])
print(mfcc_features.shape)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Character-level encoding (more flexible for unknown words)
all_text = " ".join(df["transcript"]).lower()
chars = sorted(list(set(all_text)))

char_encoder = LabelEncoder()
char_encoder.fit(list(chars))

def text_to_int_sequence(text):
    return char_encoder.transform(list(text.lower()))

# Example
encoded = text_to_int_sequence(df.iloc[0]["transcript"])
print(encoded)


In [ ]:
X = []  # features (MFCCs)
y = []  # labels (text)

for i in range(100):  # smaller subset for initial training
    mfcc = extract_mfcc(df.iloc[i]["wav_path"])
    label = text_to_int_sequence(df.iloc[i]["transcript"])
    X.append(mfcc)
    y.append(label)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X, padding="post", dtype="float32")
y_pad = pad_sequences(y, padding="post")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

model = Sequential([
    Masking(mask_value=0., input_shape=(X_pad.shape[1], X_pad.shape[2])),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(128, activation='relu'),
    Dense(len(char_encoder.classes_), activation='softmax')  # output is per character
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Use y_pad[:, 0] for simplified training (first character only)
model.fit(X_pad, y_pad[:, 0], epochs=5, batch_size=16)


In [ ]:
# Predict on one sample
pred = model.predict(X_pad[0:1])
pred_char = char_encoder.inverse_transform([np.argmax(pred[0])])
print("Predicted character:", pred_char)


In [ ]:
subset_df = df.head(2)  # Just take first 2 files

results = []
for i in range(len(subset_df)):
    audio_path = subset_df.iloc[i]['wav_path']
    try:
        result = asr(audio_path)
        results.append({
            "file": audio_path,
            "ground_truth": subset_df.iloc[i]['transcript'],
            "prediction": result["text"].strip().lower()
        })
    except Exception as e:
        print(f"Error on {audio_path}: {e}")

transcribed_df = pd.DataFrame(results)
transcribed_df.head()


In [ ]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base")


In [ ]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-tiny")


In [ ]:
from google.colab import files
uploaded = files.upload()

# Get uploaded file path
import os
audio_path = next(iter(uploaded))
